In [1]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
from pygame import mixer


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pip install pygame


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from pygame import mixer
import time

# Initialize the mixer for audio alerts
mixer.init()
mixer.music.load("C:/Users/sahan/OneDrive/Desktop/drowsiness/music.wav")

def alarm(msg):
    """Play alarm sound and print the message."""
    global alarm_status, alarm_status2

    while alarm_status:
        mixer.music.play()
        time.sleep(1)  # Avoid repeated rapid alarms

    if alarm_status2:
        mixer.music.play()
        time.sleep(1)



In [25]:
def eye_aspect_ratio(eye):
    """Calculate the Eye Aspect Ratio (EAR)."""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)


In [27]:
def final_ear(shape):
    """Calculate EAR for both eyes."""
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]

    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    ear = (leftEAR + rightEAR) / 2.0
    return ear, leftEye, rightEye



In [29]:
def lip_distance(shape):
    """Calculate the distance between the lips."""
    top_lip = np.concatenate((shape[50:53], shape[61:64]))
    low_lip = np.concatenate((shape[56:59], shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    return abs(top_mean[1] - low_mean[1])


In [31]:
# Parse arguments for webcam index
ap = argparse.ArgumentParser()
ap.add_argument("-w", "--webcam", type=int, default=0, help="index of webcam on system")
args = vars(ap.parse_args())


usage: ipykernel_launcher.py [-h] [-w WEBCAM]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\sahan\AppData\Roaming\jupyter\runtime\kernel-14fc3eaa-0d98-4ca9-83c2-ec56bb9197d8.json


SystemExit: 2

In [33]:
# Constants
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 25
YAWN_THRESH = 25



In [35]:

# Global variables
alarm_status = False
alarm_status2 = False
saying = False
COUNTER = 0


In [37]:
print("-> Loading the predictor and detector...")
detector = cv2.CascadeClassifier("C:/Users/sahan/OneDrive/Desktop/drowsiness/haarcascade_frontalface_default.xml")
predictor = dlib.shape_predictor("C:/Users/sahan/OneDrive/Desktop/drowsiness/shape_predictor_68_face_landmarks.dat")

print("-> Starting Video Stream")
vs = VideoStream(src=0).start()
time.sleep(1.0)

try:
    COUNTER = 0
    alarm_status = False
    alarm_status2 = False
    saying = False

    while True:
        # Read frame from webcam
        frame = vs.read()
        frame = imutils.resize(frame, width=450)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        rects = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30),
                                           flags=cv2.CASCADE_SCALE_IMAGE)

        for (x, y, w, h) in rects:
            rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))

            # Predict facial landmarks
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # Calculate EAR and lip distance
            ear, leftEye, rightEye = final_ear(shape)
            distance = lip_distance(shape)

 # Draw contours around eyes and lips
            cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [shape[48:60]], -1, (0, 255, 0), 1)

            # Drowsiness detection
            if ear < EYE_AR_THRESH:
                COUNTER += 1
                print (COUNTER)
                if COUNTER >= EYE_AR_CONSEC_FRAMES and not alarm_status:
                    alarm_status = True
                    Thread(target=alarm, args=("Wake up!",)).start()

                cv2.putText(frame, "DROWSINESS ALERT!", (5, 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                COUNTER = 0
                alarm_status = False

            # Yawn detection
            if distance > YAWN_THRESH:
                cv2.putText(frame, "YAWN ALERT!", (5, 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if not alarm_status2 and not saying:
                    alarm_status2 = True
                    Thread(target=alarm, args=("Take a break!",)).start()
            else:
                alarm_status2 = False

 # Display EAR and lip distance
            cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"YAWN: {distance:.2f}", (300, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Show the frame
        cv2.imshow("Frame", frame)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord("q"):
            print("Exiting...")
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    print("Cleaning up...")
    cv2.destroyAllWindows()
    vs.stop()


-> Loading the predictor and detector...
-> Starting Video Stream
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
3
4
5
6
7
8
1
1
1
1
1
1
1
1
1
1
2
3
4
5
1
2
3
1
2
3
4
5
6
7
8
9
10
11
12
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
3
1
1
1
1
1
2
3
4
5
6
7
8
1
2
3
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
1
1
1
1
2
3
4
5
1
1
1
2
3
4
5
6
7
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
3
4
5
6
7
1
1
1
1
1
1
1
1
1
1
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
1
2
3
1
1
1
1
1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
1
1
1
1
1
1
1
1
1
1
2